# Converting BEL graphs into inputs for GAT2VEC.

## Imports

In [1]:
import os

import bio2bel_phewascatalog
from GAT2VEC import paths as gat2vec_paths
from networkx import DiGraph, write_adjlist
from networkx.relabel import convert_node_labels_to_integers
from pybel import from_url, from_path
from pybel.dsl import BaseEntity, gene, protein, rna
from pybel.struct.mutation.collapse import collapse_to_genes, collapse_all_variants
from pybel_tools.assembler.reified_graph import assembler

import guilty_phewas.utils as phewas_utils

## Define the file names.

In [5]:
basedir = 'C:/Users/Mauricio/Thesis/bel_data'
url = {    
    'tau': 'https://raw.githubusercontent.com/neurommsig/neurommsig-knowledge' + \
           '/master/neurommsig_knowledge/Tau%20protein%20subgraph.bel',
    'gsk3': 'https://raw.githubusercontent.com/neurommsig/neurommsig-knowledge/' + \
           'master/neurommsig_knowledge/GSK3%20subgraph.bel',
    'alzh': 'https://raw.githubusercontent.com/neurommsig/neurommsig-knowledge/' + \
           'master/neurommsig_knowledge/GSK3%20subgraph.bel'
}

def get_base_dir(basedir, path):
    return os.path.join(basedir, path)

def get_local_bel_file(basedir, path):
    dir_ = get_base_dir(basedir, path)
    return os.path.join(dir_, path + '.bel')

def get_struct_file(basedir, path):
    dir_ = get_base_dir(basedir, path)
    return os.path.join(dir_, path + '_graph.adjlist')

def get_attr_file(basedir, path):
    dir_ = get_base_dir(basedir, path)
    return os.path.join(dir_, path + '_na.adjlist')

def get_labels_file(basedir, path):
    dir_ = get_base_dir(basedir, path)
    return os.path.join(dir_, 'labels_maped.txt')

bel = 'alzh'
# bel = 'tau'


In [6]:
# possible inputs
# url = param_dict['url']
local = get_local_bel_file(basedir, bel)

# Output files 
struct_file = get_struct_file(basedir, bel)
attr_file = get_attr_file(basedir, bel)
labels_file = get_labels_file(basedir, bel)

## Read and reify a BEL graph.

In [7]:
# graph = from_url(url[bel])
graph = from_path(local)

print("Starting nodes", len(graph.nodes))
print("Starting edges", len(graph.edges))

# Collapse all variants is removing also the pmod(X) from the BaseEntity
collapse_all_variants(graph)
# TODO collapse_to_genes removes the ptm information before converting
# collapse_to_genes(graph)

print("Nodes after collapse", len(graph.nodes))
print("Edges after collapse", len(graph.edges))

# Reify the edges
rbg = assembler.reify_bel_graph(graph)

print("Nodes after edge reification", len(rbg.nodes))
print("Edges after edge reification", len(rbg.edges))

ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\alzh\alzh.bel:35457:23 MissingNamespaceNameWarning "Glycoside hydrolases" is not in the GFAM namespace
ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\alzh\alzh.bel:44994:2 MissingNamespaceNameWarning "N-terminal domain of APP" is not in the HBP namespace
ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\alzh\alzh.bel:44995:2 MissingNamespaceNameWarning "N-terminal domain of APP" is not in the HBP namespace
ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\alzh\alzh.bel:44996:2 MissingNamespaceNameWarning "N-terminal domain of APP" is not in the HBP namespace
ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\alzh\alzh.bel:44997:2 MissingNamespaceNameWarning "N-terminal domain of APP" is not in the HBP namespace
ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\alzh\alzh.bel:44998:2 MissingNamespaceNameWarning "N-terminal domain of APP" is not in the HBP namespace
ERROR:pybel.parser:C:/Users/Mauricio/Thesis/bel_data\a

Starting nodes 5165
Starting edges 17226
Nodes after collapse 4722
Edges after collapse 16675


Nodes after edge reification 20720
Edges after edge reification 32266


## Assess the converted entity and predicate nodes.

In [9]:
qty_predicate = {}
qty_prot, qty_rna, qty_gene = 0, 0, 0
for i in rbg.nodes:
    if isinstance(i, BaseEntity):
        if isinstance(i, rna):
            qty_rna += 1
        elif isinstance(i, protein):
            qty_prot += 1
        elif isinstance(i, gene):
            qty_gene += 1
    else:
        if rbg.nodes[i]['label'] in qty_predicate:
            qty_predicate[rbg.nodes[i]['label']] += 1
        else:
            qty_predicate[rbg.nodes[i]['label']] = 1

print("Predicates")
print(qty_predicate)

print(f"Proteins {qty_prot}")
print(f"RNA      {qty_rna}")
print(f"Gene     {qty_gene}")

Predicates
{'correlatesTo': 7686, 'abundance': 5491, 'degradates': 156, 'activates': 640, 'translates': 490, 'hasComponent': 1670}
Proteins 1496
RNA      492
Gene     493


## Adding PheWAS annotation (as attributes).

### Load the links between Phenotype and genes

In [10]:
phewas_manager = bio2bel_phewascatalog.Manager()
pw_dict = phewas_manager.to_dict()

PheWAS Catalog - generating Dict: 100%|█████████████████████████████████████| 215107/215107 [00:19<00:00, 10775.18it/s]


### Save the disease list into the graph

In [12]:
start_ind = len(rbg.nodes)
phenotypes_list = set([phe for _list in pw_dict.values()
                           for odds, phe in _list])
# unique_phenotypes = set().union(*pw_dict.values())
enum_phenotypes = enumerate(phenotypes_list, start=start_ind)
att_mappings = {phenot: num for num, phenot in enum_phenotypes}

annotated_graph = rbg.copy()
phewas_utils.add_disease_attribute(annotated_graph, pw_dict)

## Genarate the labels for classification.

In [13]:
phewas_utils.download_for_disease('EFO_0000249', 'ot_entrez.txt')
targets = phewas_utils.parse_gene_list('ot_entrez.txt', rbg)
print('Targets found in the network:', len(targets))

querying 1-146...done.
Targets found in the network: 91


### Targets not present at the network

In [14]:
with open('ot_entrez.txt') as f:
    alltargets = set(f.read().split())
print(alltargets.difference(targets))
# Targets found in the network: 86, without taking non causal correlations
# Targets found in the network: 91, taking non causal correlations
# missing targets were:
"""'IMPA1', 'ADORA3', 'PDE3A', 'ADRA2A', 'GNRHR', 'ADRA1B', 'HTR3A', 'HTR3D', 'SCN8A', 'APH1B', 'CSF2RA', 'FAAH', 'ADRA2C', 'MAOA', 'SCN2A', 'ALDH5A1', 'AGTR1', 'DDB1', 'DRD3', 'SCN11A', 'SCN1A', 'HTR3C', 'VKORC1', 'CACNA1B', 'SV2A', 'SCN5A', 'PAH', 'SCN3A', 'SCN9A', 'SLC6A2', 'ADORA1', 'FGFR3', 'HTR2C', 'HTR3B', 'PGR', 'RXRB', 'ADRA1A', 'SCN4A', 'DRD2', 'ADORA2B', 'SCN10A', 'CUL4A', 'SLC6A3', 'RARG', 'RBX1', 'PPARD', 'IFNAR2', 'SLC5A2', 'ADRA1D', 'ADRA2B', 'HTR3E', 'MAOB', 'CRBN', 'KIT', 'RARA', 'ATP4A', 'HRH3', 'CACNA2D1', 'SCN7A', 'SLC6A4'""";

{'SCN11A', 'ADRA2C', 'HTR3B', 'MAOB', 'PDE3A', 'SCN8A', 'CUL4A', 'SCN10A', 'HTR2C', 'PGR', 'SLC6A2', 'CACNA1B', 'GNRHR', 'SCN1A', 'HTR3E', 'RBX1', 'SLC6A3', 'HTR3A', 'SCN5A', 'SLC6A4', 'DDB1', 'SLC5A2', 'ATP4A', 'HTR3D', 'DRD2', 'SCN7A', 'ADRA1A', 'PPARD', 'VKORC1', 'ALDH5A1', 'MAOA', 'ADRA1D', 'ADORA2B', 'ADORA1', 'DRD3', 'HTR3C', 'RARG', 'RARA', 'IFNAR2', 'ADRA2B', 'PAH', 'CRBN', 'ADRA1B', 'SCN9A', 'CACNA2D1', 'KIT', 'ADRA2A', 'HRH3', 'SCN2A', 'SCN3A', 'RXRB', 'SCN4A', 'ADORA3', 'CSF2RA', 'IMPA1'}


## Create adjacency list and labels files for GAT2VEC

In [18]:
import logging

# Structure graph
out_rbg = convert_node_labels_to_integers(annotated_graph, 
                                          # first_label=1, 
                                          label_attribute='old_label')
write_adjlist(out_rbg, struct_file)

# print(att_mappings)
# Attribute graph
phewas_utils.write_adj_file_attribute(out_rbg, attr_file, att_mappings)


In [19]:
phewas_utils.write_labels_file(labels_file, out_rbg, targets)